# Scrap de dados

Este notebook tem como objetivo fazer um scrap de dados a fim de obter novos dados para complementar o dataset original.

#### Motivacional

Como os dados do produto do dataset original continham muita pouca informação sobre o produto, como proposta de enriquecimento, foi utilizado o processo de scrap de dados para obter informações da página da [Ambev](https://www.ambev.com.br) que contém informações mais detalhadas sobre alguns produtos.

#### Implementação

Para a proposta inicial a ideia era utilizar uma requisição HTTP básica para o endereço, no entanto pela forma que a página esta desenvolvida o retorno da requisição retorna um java script indicando que a página utiliza algum framework web como: Angular, React ou Vue sendo um SPA (Single Page Application). Esse formato de aplicação utiliza o retorno no formato javascript para renderizar a pagina em tempo de execução. Essa abordagem então impossibilita a utilização da primeira proposta.

Para solucionar este ponto, foi utilizado o Selenium para simular o acesso real de um usuário, através do browser, e assim ter acesso ao html renderizado após o processo de acesso à página.

#### Bibliotecas

Para resolver esses tipos de desafios o Selenium normalmente é a tecnologia mais como a ser aplicada. Todo o processamento restante foi feito utilizando-se as bibliotecas padrões do proprio Python.

### Importação

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings("ignore")

### Funções auxiliares

As funções abaixo tem o objetivo de obter os dados do HTML retornado pela pagina

In [2]:
def get_grid_item(element_name = 'elements-grid__items'):
    try:
        return driver.find_element_by_class_name(element_name)
    except:
        return None

In [3]:
def get_links(element_ul) -> list:
    links = []
    for li in element_ul.find_elements_by_tag_name('li'):
        a = li.find_element_by_tag_name('a')
        links.append(a.get_attribute('href'))
    return links

A função baixo cria o dicionario com os dados extraidos da pagina

In [4]:
def create_info(section_info, complements, url) -> dict:
    
    if len(section_info) > 5:
        descricao, ingredientes, alergicos = section_info[2:5]
    else:
        descricao = None
        ingredientes = None
        alergicos = None

    if len(complements) > 5:
        harmonizacao, _, _, temperatura, ibu = complements[0:5]
    elif len(complements) > 3:
        _, temperatura, ibu = complements[0:3]
        harmonizacao = None
    else:
        harmonizacao = None
        temperatura = None
        ibu = None

    info = { 'descricao': descricao, 'ingredientes': ingredientes.replace('INGREDIENTES', '') if ingredientes else None, 'alergicos': alergicos.replace('ALERGÊNICOS', '') if alergicos else None  }
    info['url'] = url
    if harmonizacao and 'Harmonização' in harmonizacao:
        info['harmonizacao'] =  harmonizacao.splitlines()[1:][0]
    else:
        info['harmonizacao'] = None
    
    if temperatura and 'Temperatura' in temperatura:
        info['temperatura'] = temperatura.splitlines()[1:][0]
    else:
        info['temperatura'] = None
    
    if ibu and 'IBU' in ibu:
        info['ibu'] = ibu.splitlines()[1:][0] if ibu else None
    else:
        info['ibu'] = None
        
    return info

Listagem com todos os dados extraidos da página

In [5]:
dados = []

### Preparação do scrap

In [6]:

options = Options()
#options.add_argument('--headless')
options.add_argument("--start-maximized")

### Workflow para obteção dos dados

In [12]:
with webdriver.Chrome('C:\drive\chromedriver.exe', options=options) as driver:
    driver.get('https://www.ambev.com.br/marcas/cervejas/')

    maior_idade = driver.find_element_by_class_name('age-gate-dialog__answer-button')
    maior_idade.click()

    ul = get_grid_item()
    beers = get_links(ul)
    
    for beer in beers:

        if any(filter(lambda x: beer in x['url'], dados)):
            continue

        driver.get(beer)

        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.END)
        time.sleep(2)

        print('URL: ', beer)

        n_beers = get_grid_item()
        if (n_beers):
            links = get_links(n_beers)
            beers.extend([link for link in links if link not in beers])
            continue
    
        section_info = driver.find_element_by_xpath("//div[5]/section/div[1]")
        infos = section_info.text.splitlines()
        complements = [e.text for e in driver.find_elements_by_xpath('//div[5]/section/div[3]/div/p') if e.text != '']

        dados.append(create_info(infos, complements, beer))

URL:  https://www.ambev.com.br/marcas/cervejas/adriatica
URL:  https://www.ambev.com.br/marcas/cervejas/andes
URL:  https://www.ambev.com.br/marcas/cervejas/antarctica
URL:  https://www.ambev.com.br/marcas/cervejas/becks
URL:  https://www.ambev.com.br/marcas/cervejas/berrio-do-piaui
URL:  https://www.ambev.com.br/marcas/cervejas/bohemia
URL:  https://www.ambev.com.br/marcas/cervejas/brahma
URL:  https://www.ambev.com.br/marcas/cervejas/budweiser
URL:  https://www.ambev.com.br/marcas/cervejas/caracu
URL:  https://www.ambev.com.br/marcas/cervejas/cervejaria-colorado
URL:  https://www.ambev.com.br/marcas/cervejas/corona
URL:  https://www.ambev.com.br/marcas/cervejas/esmera-de-goias
URL:  https://www.ambev.com.br/marcas/cervejas/franziskaner
URL:  https://www.ambev.com.br/marcas/cervejas/goose-island
URL:  https://www.ambev.com.br/marcas/cervejas/hoegaarden
URL:  https://www.ambev.com.br/marcas/cervejas/kona
URL:  https://www.ambev.com.br/marcas/cervejas/leffe
URL:  https://www.ambev.com.b

### Exibição dos dados carregados

In [13]:
dados

[{'descricao': 'A Cerveja Adriática 600ml foi criada pelo alemão Henrique Thielen, um visionário cervejeiro do início do século XX. Ela teve seu nome em homenagem à cervejaria que traduz toda uma era de tradição passada de pai para filho. Hoje, conhecida como a irmã mais velha da Original, ela é uma pedida certa para a mesa de bar. Reconhecidamente uma cerveja puro malte de alta qualidade, fácil de beber e com aromas especiais que dão um toque equilibrado.',
  'ingredientes': ': Água, malte e lúpulo.',
  'alergicos': ': Contém cevada e glúten.',
  'url': 'https://www.ambev.com.br/marcas/cervejas/adriatica',
  'harmonizacao': None,
  'temperatura': None,
  'ibu': None},
 {'descricao': 'Puro malte argentino inspirado no frescor das Cordilheiras dos Andes. Cerveja mais fresca, inspirada nas cordilheiras. Ingredientes argentinos: malte e lúpulo argentinos.',
  'ingredientes': ': Água, malte e lúpulo.',
  'alergicos': ': Contém cevada e glúten.',
  'url': 'https://www.ambev.com.br/marcas/ce

### Persistência das informações obtidas

In [14]:
import json

with open('produto_caracteristica.json', 'w') as file:
    file.write(json.dumps(dados))